In [1]:
!pip install requests

You should consider upgrading via the 'C:\Users\ariel\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [2]:
import base64
import requests
import datetime

In [3]:
from urllib.parse import urlencode

In [4]:
client_id = 'a7f54a243bf44f1f8930ce153a1821f7'
client_secret = 'bb0c65ae068a4c2487c81fe326646cc1'

In [5]:
class SpotifyAPI(object):
    access_token = None
    access_token_expires = datetime.datetime.now()
    access_token_did_expire = True
    client_id = None
    client_secret = None
    token_url = "https://accounts.spotify.com/api/token"
    
    def __init__(self, client_id, client_secret, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.client_id = client_id
        self.client_secret = client_secret

    def get_client_credentials(self):
        """
        Returns a base64 encoded string
        """
        client_id = self.client_id
        client_secret = self.client_secret
        if client_secret == None or client_id == None:
            raise Exception("You must set client_id and client_secret")
        client_creds = f"{client_id}:{client_secret}"
        client_creds_b64 = base64.b64encode(client_creds.encode())
        return client_creds_b64.decode()
    
    def get_token_headers(self):
        client_creds_b64 = self.get_client_credentials()
        return {
            "Authorization": f"Basic {client_creds_b64}"
        }
    
    def get_token_data(self):
        return {
            "grant_type": "client_credentials"
        } 
    
    def perform_auth(self):
        token_url = self.token_url
        token_data = self.get_token_data()
        token_headers = self.get_token_headers()
        r = requests.post(token_url, data=token_data, headers=token_headers)
        if r.status_code not in range(200, 299):
            return False
        data = r.json()
        now = datetime.datetime.now()
        access_token = data['access_token']
        expires_in = data['expires_in'] # seconds
        expires = now + datetime.timedelta(seconds=expires_in)
        self.access_token = access_token
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now
        return True

    def get_access_token(self):
        access_token = self.access_token
        expires = self.access_token_expires
        now = datetime.datetime.now()

        if expires < now or access_token == None:
            self.perform_auth()
            access_token = self.access_token
            return access_token
            
        return access_token

    def get_required_header(self):
        access_token = self.get_access_token()
        required_header = {
            "Authorization": f"Bearer {access_token}"
        }
        return required_header

    def get_specific_request_line(self, _id, version = 'v1', request_type = 'albums'):
        url = f"https://api.spotify.com/{version}/{request_type}/{_id}"
        request_line = requests.get(url, headers = self.get_required_header())
        if request_line.status_code not in range (200,299):
            return {}
        return request_line.json()
    
    def get_album(self, _id): 
        return self.get_specific_request_line(_id, request_type = 'albums')

    def get_artist(self, _id):
        return self.get_specific_request_line(_id, request_type = 'artists')
    
    def base_query(self, query_params):
        url = " https://api.spotify.com/v1/search"
        lookup_url = f"{url}?{query_params}"
        request_line = requests.get(lookup_url, headers = self.get_required_header())
        if request_line.status_code not in range (200,299):
            return {}
        return request_line.json()


    def search(self, query = None, operator = None, operator_query = None, search_type = "track"):
        if query == None:
            raise Exception ("Query can't be none")

        if isinstance (query, dict):
            " ".join([f"{k}:{v}" for k,v in query.items()])

        if operator != None and operator_query != None:
            if operator == "OR" or operator == "NOT":
                operator.upper()
                if  isinstance(operator_query, str):
                    query = (query, operator, operator_query)

        query_params = urlencode({
        "q":f"{query}", "type":f"{search_type}"
        }) 
        return self.base_query(query_params)


In [6]:
spotify = SpotifyAPI(client_id, client_secret)

In [7]:
spotify.search("positions", search_type = "track")

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=positions&type=track&offset=0&limit=20',
  'items': [{'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/66CXWjxzNUsdJxJ2JdwvnR'},
       'href': 'https://api.spotify.com/v1/artists/66CXWjxzNUsdJxJ2JdwvnR',
       'id': '66CXWjxzNUsdJxJ2JdwvnR',
       'name': 'Ariana Grande',
       'type': 'artist',
       'uri': 'spotify:artist:66CXWjxzNUsdJxJ2JdwvnR'}],
     'available_markets': ['AD',
      'AE',
      'AG',
      'AL',
      'AM',
      'AO',
      'AR',
      'AT',
      'AU',
      'AZ',
      'BA',
      'BB',
      'BD',
      'BE',
      'BF',
      'BG',
      'BH',
      'BI',
      'BJ',
      'BN',
      'BO',
      'BR',
      'BS',
      'BT',
      'BW',
      'BY',
      'BZ',
      'CA',
      'CH',
      'CI',
      'CL',
      'CM',
      'CO',
      'CR',
      'CV',
      'CW',
      'CY',
      'CZ',
      'DE',
      'DJ',
      'DK',
    

In [8]:
spotify.search(query="God is a", operator="NOT", operator_query='woman', search_type="track")

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=%28%27God+is+a%27%2C+%27NOT%27%2C+%27woman%27%29&type=track&offset=0&limit=20',
  'items': [{'album': {'album_type': 'compilation',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/0LyfQWJT6nXafLPZqxe9Of'},
       'href': 'https://api.spotify.com/v1/artists/0LyfQWJT6nXafLPZqxe9Of',
       'id': '0LyfQWJT6nXafLPZqxe9Of',
       'name': 'Various Artists',
       'type': 'artist',
       'uri': 'spotify:artist:0LyfQWJT6nXafLPZqxe9Of'}],
     'available_markets': ['AD',
      'AE',
      'AG',
      'AL',
      'AM',
      'AO',
      'AR',
      'AT',
      'AU',
      'AZ',
      'BA',
      'BB',
      'BD',
      'BE',
      'BF',
      'BG',
      'BH',
      'BI',
      'BJ',
      'BN',
      'BO',
      'BR',
      'BS',
      'BT',
      'BW',
      'BY',
      'BZ',
      'CA',
      'CH',
      'CI',
      'CL',
      'CM',
      'CO',
      'CR',
      'CV',
      'CW',
      'CY',
     